Initiliase les lignes et regarde les titres les plus récurrents

In [4]:
import matplotlib.pyplot as plt

with open("../fichier_sans_lignes_vides.txt", "r", encoding="utf8") as f:
    lines = f.readlines()

lines = [line.strip() for line in lines if line.strip()]

d_title = {}
for i in range(len(lines)):
    if lines[i][0:5] == "<text":
        p = ""
        boo = False
        for j in range(len(lines[i])):
            if lines[i][j-7:j] == 'title="':
                boo = True
            if lines[i][j] == '"':
                boo = False
            if boo != False:
                p += lines[i][j]
        if p in d_title:  #ajoute dans un dictionnaire le thème et son itération correspondante dans une liste
            d_title[p].append(i)
        else:
            d_title[p] = [i]
    if i > 50:
        break 
# print(d_title)

# for elt in d_title:
#     if len(d_title[elt]) > 1:
#         print(elt,"---",len(d_title[elt]))  
#affiche le nombre de thèmes qui apparaisse + de 2 fois, et c'est souvent 2 fois max donc pas fou de faire une analyse sur les thèmes sauf si on les généralise mais c'est risqué sur des questions précises
# pas concluant

Regarde les questions et les mets dans un dictionnaire en fonction d'un département

In [7]:
# d_dept_quest = {}
# for i in range(len(lines)):
#     if lines[i][0:5] == "<text":
#         p = ""
#         boo = False
#         for j in range(len(lines[i])):
#             # if lines[i][j-12:j] == 'department="': #par nom de departement
#             # if lines[i][j-8:j] == 'region="':  #par region
#             if lines[i][j-15:j] == 'departmentiso="':  #par numéro de departement
#                 boo = True
#             if lines[i][j] == '"':
#                 boo = False
#             if boo != False:
#                 p += lines[i][j]
#         if p in d_dept_quest:  #ajoute dans un dictionnaire le thème et son itération correspondante dans une liste
#             # print(lines[i])
#             d_dept_quest[p].append(lines[i])
#         else:
#             d_dept_quest[p] = [lines[i]]
    # if i > 100:
    #     break

   # for elt in d_dept_quest:
#     d_dept_quest[elt] = len(d_dept_quest[elt])
#     print(elt,"---",d_dept_quest[elt]) 
# print("\n\n")

La même chose en faisant les réponses

In [5]:
d_dept_rep = {}
for i in range(len(lines)):
    if lines[i][0:5] == "<text":
        p = ""
        boo = False
        for j in range(len(lines[i])):
            # if lines[i][j-12:j] == 'department="': #par nom de departement
            # if lines[i][j-8:j] == 'region="':  #par region
            if lines[i][j-15:j] == 'departmentiso="':  #par numéro de departement
                boo = True
            if lines[i][j] == '"':
                boo = False
            if boo != False:
                p += lines[i][j]
    else:
        if lines[i] != "</text>":
            if p in d_dept_rep:  #ajoute dans un dictionnaire le thème et son itération correspondante dans une liste
                # print(lines[i])
                d_dept_rep[p].append(lines[i])
            else:
                d_dept_rep[p] = [lines[i]]
    if i > 100:
        break

list_dep = ["FR-09","FR-11","FR-12","FR-30","FR-31","FR-32","FR-34","FR-46","FR-48","FR-65","FR-66","FR-81","FR-82"]
for cle in list(d_dept_rep.keys()):
    if cle not in list_dep:
        del d_dept_rep[cle]
# for elt in d_dept_rep:
#     d_dept_rep[elt] = len(d_dept_rep[elt])
#     print(elt,"---",d_dept_rep[elt]) 
#par départment c'est déjà mieux (données sur 100 départements)

#trier de manière décroissante
sorted_d = dict(sorted(d_dept_rep.items(), key=lambda x: len(x[1]), reverse=True))
print(sorted_d)

{'FR-31': ["Informer sur les grands postes de dépenses du budget de l'état. Les gens n'ont pas idée que les grands postes ce sont les salaires des fonctionnaires, et les prestations sociales. Ils ont l'impression que des gens se gavent avec leur argent.", "Il faudrait faire un équivalent à ce qui se fait aux USA: Tout citoyen français est redevable d'impôt en France. Si il réside à l'étranger, il paie ses impôts localement dans son pays de résidence, mais fait une déclaration aussi en France. Son impôt est calculé comme si il résidait en France. Si le montant est supérieur à ce qu'il paie dans le pays de résidence, alors il paie à la France le complément: (impôt qu'il paierait si résident français - impôt payé dans pays de résidence). Avec ce système, il devient inutile de se domicilier à l’étranger (Belgique, Monaco, Portugal,...) pour échapper à l’impôt français, car on n'y gagne plus rien. Si ce principe n'est pas respecté, la personne perd sa citoyenneté française, et devra prendre

In [6]:
def rep_dep(num_dep):
    return sorted_d["FR-"+str(num_dep)]

print(rep_dep(34))

["Faire reposer l'impôt sur le revenu sur la totalité des rentrées , donc y intégrer tous les revenus de remplacement ( allocations , par exemple) ; bien sûr modifier en conséquence le barême.... Il est malsain de rendre l'impôt indolore (CSG ,prélèvement à la source ) ; cela déresponsabilise le citoyen", "Augmenter le temps de travail|Revoir les conditions d'attribution de certaines aides sociales", 'Les aides au logement, soit directes ,soit indirectes doivent être mieux ciblées sur ceux qui en ont vraiment besoin .Les alloc.logement profitent surtout aux loueurs qui augmentent les loyers . Les logements sociaux ne sont pas toujours occupés par de vrais ayant-droits .']
